In [ ]:
import torch
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile

In [ ]:
from torch import nn

In [ ]:
class Classification(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential( #1, 256, 256
            nn.Conv2d(1, 16, 4, 2, 1), #16, 128, 128
            nn.ReLU(),
            nn.Conv2d(16, 32, 4, 2, 1), #32, 64, 64
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1), #64, 32, 32
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1), #128, 16, 16
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, 2, 1), #256, 8, 8
            nn.ReLU(),
            nn.Flatten(), #256*8*8
            nn.Linear(256*8*8, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 10),
            nn.Softmax(1),
        )
    def forward(self, images):
        labels = self.net(images)
        return labels

In [ ]:
final_epoch = 400

In [ ]:
C = Classification()
C.load_state_dict(torch.load(f'C_{final_epoch}.pth'))

In [ ]:
model = C
model.eval()
example = torch.rand(1, 1, 256, 256)
traced_scipt_module = torch.jit.trace(model, example)
traced_scipt_module_optimized = optimize_for_mobile(traced_scipt_module)
traced_scipt_module_optimized._save_for_lite_interpreter('module.ptl')
